In [10]:
#solve wiiplay find mii game_1 : "find_this_mii"

import cv2
import numpy as np

scaling_factor = 1
cap = cv2.VideoCapture("wiiplay.mp4")
frame_seq = 4820

cap.set(cv2.CAP_PROP_POS_FRAMES , frame_seq)
_, first_frame = cap.read()
first_frame = cv2.resize(first_frame, None, fx=0.8, fy=0.8, interpolation=cv2.INTER_AREA)
bbox = cv2.selectROI(first_frame, False, False)
cv2.destroyAllWindows()
x, y, w, h = bbox
template = np.copy(first_frame[y:y+h, x:x+w])

while True:
    frame_seq += 1
    if frame_seq > 5000:
        frame_seq = 4820
    cap.set(cv2.CAP_PROP_POS_FRAMES , frame_seq)
    status_cap, frame0 = cap.read()
    if not status_cap:
        break
    frame = cv2.resize(frame0, None, fx=scaling_factor, fy=scaling_factor)
    
    # Do simple template matching
    result = cv2.matchTemplate(frame, template, cv2.TM_CCOEFF_NORMED)
    result = cv2.normalize(result, None, 0, 1, cv2.NORM_MINMAX)
    minVal, maxVal, minLoc, maxLoc = cv2.minMaxLoc(result)
    
    cv2.rectangle(frame, (maxLoc[0], maxLoc[1]), 
                (maxLoc[0] + int(w), maxLoc[1] + int(h)),(0, 0, 255), 3)
    
    cv2.imshow("find_this_mii", frame)
    k = cv2.waitKey(1)
    if k == 27:
        break
        
cap.release()
cv2.destroyAllWindows()
'''please choose suitable template for optimal result'''

'please choose suitable template for optimal result'

In [11]:
#solve wiiplay find mii game_2 : "find_two_look_alike"
#requires haarcascade_frontalface_default.xml
import cv2
import numpy as np

scaling_factor = 1
cap = cv2.VideoCapture("wiiplay.mp4")
frame_seq = 2180

# Using HOG for pedestrian detection
hog = cv2.HOGDescriptor()
hog.setSVMDetector(cv2.HOGDescriptor_getDefaultPeopleDetector())

# Using haarcascade for face detection
face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
final_template = None
max_similarity = -1
templates = list()

while True:
    frame_seq += 1
    if frame_seq > 2380:
        frame_seq = 2180
    cap.set(cv2.CAP_PROP_POS_FRAMES , frame_seq)
    status_cap, frame0 = cap.read()
    if not status_cap:
        break
    frame = cv2.resize(frame0, None, fx=0.75, fy=0.75)
    final_frame = frame.copy()
    
    # Draw pedestrian
    locations, weights = hog.detectMultiScale(frame, finalThreshold = 2)
    for loc in locations:   
        cv2.rectangle(final_frame, (loc[0], loc[1]-50),(loc[0]+loc[2], loc[1]+loc[3]), (0, 255, 0), 3)
    
    # Face detection
    face_rects = face_cascade.detectMultiScale(frame, scaleFactor=1.5, minNeighbors=1)
    # Draw face (strict criteria) in dark blue and append face templates
    for (x,y,w,h) in face_rects:
        hsv_face = cv2.cvtColor(frame[y+5:y-10+h, x+5:x-10+w], cv2.COLOR_BGR2HSV)
        # Check hsv of detected faces, if pass then put into templates list
        if (np.mean(hsv_face[:,:,0])>=0 and np.mean(hsv_face[:,:,0])<=35):
            if (np.mean(hsv_face[:,:,1])>=50 and np.mean(hsv_face[:,:,1])<=200):
                if (np.mean(hsv_face[:,:,2])>=60 and np.mean(hsv_face[:,:,2])<=235): 
                    templates.append(frame[y+5:y-10+h, x+5:x-10+w])
                    cv2.rectangle(final_frame, (x-5,y-5), (x+w+5,y+h+5), (255,0,0), 3)
    # Draw face (weak criteria) in light blue
    face_rects = face_cascade.detectMultiScale(frame, scaleFactor=1.05, minNeighbors=3)
    for (x,y,w,h) in face_rects:
        cv2.rectangle(final_frame, (x-5,y-5), (x+w+5,y+h+5), (255,128,0), 3)

        
    index = 0
    for temp in templates:
        result = cv2.matchTemplate(frame, temp, cv2.TM_CCOEFF_NORMED)
        result = cv2.normalize(result, None, 0, 1, cv2.NORM_MINMAX)
        minVal, maxVal, minLoc, maxLoc = cv2.minMaxLoc(result)
        # Highest similarity template is put to final_template
        if (maxVal >= max_similarity) and maxVal > 0.99:
            final_template = temp.copy()
            templates[index] = temp
            index+=1
    # Only keep first 3 templates in front of the list (top 3 highest similarity)
    del templates[3:]
    
    # Draw similar faces using final_template template matching
    if (templates):
        result = cv2.matchTemplate(frame, final_template, cv2.TM_CCOEFF_NORMED)
        result = cv2.normalize(result, None, 0, 1, cv2.NORM_MINMAX)
        minVal, maxVal, minLoc, maxLoc = cv2.minMaxLoc(result)
        result2 = np.reshape(result, result.shape[0]*result.shape[1])
        sort = np.argsort(result2)[::-1]
        (y1, x1) = np.unravel_index(sort[0], result.shape) # best match
        for i in range (len(sort)):
            (y, x) = np.unravel_index(sort[i], result.shape)
            if (abs(y-y1)>10 and abs(x-x1)>10): # if position of second best match is 25 pxs away (no overlapping)
                (y2, x2) = np.unravel_index(sort[i], result.shape) # second best match
                break
        cv2.rectangle(final_frame, (x1-5, y1-5), (x1 + int(w)-5, y1 + int(h)-5),(0, 0, 255), 3)
        cv2.rectangle(final_frame, (x2-5, y2-5), (x2 + int(w)-5, y2 + int(h)-5),(0, 0, 255), 3)
    
    # Additional show current template being used to find similar faces
    if (templates): 
        final_template = cv2.resize(final_template, (256, 256), fx=1, fy=1)
        cv2.imshow('Looking for', final_template)
    cv2.imshow("find_two_look_alike", final_frame)
    k = cv2.waitKey(1)
    if k == 27:
        break
        
cap.release()
cv2.destroyAllWindows()
'''reminder: requires haarcascade_frontalface_default.xml'''

'reminder: requires haarcascade_frontalface_default.xml'

In [12]:
#solve wiiplay find mii game_3 : "find_the_fastest_character"
# pip install opencv-contrib-python --user

import cv2
import numpy as np

scaling_factor = 1
cap = cv2.VideoCapture("wiiplay.mp4")
frame_seq = 2480

cap.set(cv2.CAP_PROP_POS_FRAMES , frame_seq)
_, first_frame = cap.read()

# Using HOG for pedestrian detection
hog = cv2.HOGDescriptor()
hog.setSVMDetector(cv2.HOGDescriptor_getDefaultPeopleDetector())
locations, weights = hog.detectMultiScale(first_frame, hitThreshold = 0.1, winStride=(5,10)
                                          ,finalThreshold = 1)

# Create bbox for each detected pedestrian
prev_pos = list()
bboxes = list()
for loc in locations:    
    cv2.rectangle(first_frame, (loc[0], loc[1]),
                  (loc[0]+loc[2], loc[1]+loc[3]), (255, 0, 0), 3)
    #bboxes.append((loc[0]+30, loc[1]+30,loc[2]-40, loc[3]-40))
    bboxes.append((loc[0]+int(loc[2]/3), loc[1]+int(loc[3]/4), int(loc[2]/3), int(loc[3]/7)))
    prev_pos.append((loc[0], loc[1]))
cv2.imshow('first frame', first_frame)
cv2.waitKey()

# Use median flow to track each bbox
status_trackers = list()
trackers = list()
for bbox in bboxes:
    trackers.append(cv2.TrackerMedianFlow_create())
for i in range(len(trackers)):
    status_trackers.append(trackers[i].init(first_frame, bboxes[i]))

while True:
    frame_seq += 1
    if frame_seq > 2600:
        frame_seq = 2480
    cap.set(cv2.CAP_PROP_POS_FRAMES , frame_seq)
    status_cap, frame0 = cap.read()
    if not status_cap:
        break
    frame = cv2.resize(frame0, None, fx=scaling_factor, fy=scaling_factor)
    
    max_pos_diff = -1
    # Update each bbox tracker and find maximum position difference
    for i in range (len(status_trackers)):
        if status_trackers[i]:
            status_trackers[i], bboxes[i] = trackers[i].update(frame)
        if status_trackers[i]:
            x, y, w, h = [int(k) for k in bboxes[i]]
            cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 3)
            if (abs(x-prev_pos[i][0])+abs(y-prev_pos[i][1])>max_pos_diff):
                max_pos = prev_pos[i]
            prev_pos[i] = (x, y)

    # Draw red rectangle on tracker with maximum position difference between frames
    cv2.rectangle(frame, (max_pos[0], max_pos[1]), (max_pos[0]+100, max_pos[1]+100), (0,0,255), 3)
    cv2.imshow("find_the_fastest_character", frame)
    k = cv2.waitKey(1)
    if k == 27:
        break
        
cap.release()
cv2.destroyAllWindows()
'''please press Enter or any key to continue after first frame'''
'''reminder: pip install opencv-contrib-python --user'''

'reminder: pip install opencv-contrib-python --user'

In [13]:
#solve wiiplay find mii game_4 : "find_two_odds"

import cv2
import numpy as np

scaling_factor = 1
cap = cv2.VideoCapture("wiiplay.mp4")
frame_seq = 1650
cap.set(cv2.CAP_PROP_POS_FRAMES , frame_seq)
_ , prev_frame = cap.read()
prev_frame = cv2.cvtColor(prev_frame, cv2.COLOR_BGR2GRAY)
prev_frame = cv2.resize(prev_frame, (0,0), None, fx=scaling_factor, fy=scaling_factor)
first_frame = True

face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

def display_flow(img, flow, stride=10):
    left = 0
    right = 0
    for index in np.ndindex(flow[::stride, ::stride].shape[:2]):
        pt1 = tuple(i*stride for i in index)
        delta = flow[pt1].astype(np.int32)[::-1]
        pt2 = tuple(pt1 + 2*delta)
        
        if 2 <= cv2.norm(delta) <= 10:
            cv2.arrowedLine(img, pt1[::-1], pt2[::-1], (255,0,0), 1, cv2.LINE_AA, 0, 0.1)
        if (pt1[1]-pt2[1])>0: # Sum of left arrows
            left += 1
        elif (pt2[1]-pt1[1])>0: # Sum of right arrows
            right += 1
            
    # Get positions of faces that have odd arrows
    face_rects = face_cascade.detectMultiScale(frame, scaleFactor=1.1, minNeighbors=3)
    # Add a new column in face_rects for voting
    N = len(face_rects)
    face_rects = np.c_[face_rects, np.zeros(N)]
    face_rects = face_rects.astype(int)
    for index in np.ndindex(flow[::stride, ::stride].shape[:2]):
        pt1 = tuple(i*stride for i in index)
        delta = flow[pt1].astype(np.int32)[::-1]
        pt2 = tuple(pt1 + 2*delta)
        for index, (x,y,w,h,votes) in enumerate(face_rects):
            if (left>right) and (left+right)>500 and pt2[1]-pt1[1]>0 and (pt2[1]>x and pt2[1]<x+w) and (pt2[0]>y and pt2[0]<y+h):
                face_rects[index,4]+=1
                break
            elif (right>left) and (left+right)>500 and (pt1[1]-pt2[1])>0 and (pt1[1]>x and pt1[1]<x+w) and (pt1[0]>y and pt1[0]<y+h):
                face_rects[index,4]+=1
                break
            cv2.rectangle(frame, (x,y), (x+w,y+h), (255,0,0), 3)
    for (x,y,w,h,votes) in face_rects:
        if (votes > 5):
            cv2.rectangle(frame, (x,y), (x+w,y+h), (0,0,255), 3)
    #norm_opt_flow = np.linalg.norm(flow, axis=2)
    #norm_opt_flow = cv2.normalize(norm_opt_flow, None, 0, 1, cv2.NORM_MINMAX)
    #cv2.imshow('optical flow', img)
    #cv2.imshow('optical flow magnitude', norm_opt_flow)
    k = cv2.waitKey(1)
    if k == 27:
        return 1
    else:
        return 0

    
bboxes = list()
while True:
    frame_seq += 1
    if frame_seq > 1800:
        frame_seq = 1650
    cap.set(cv2.CAP_PROP_POS_FRAMES , frame_seq)
    status_cap, frame0 = cap.read()
    if not status_cap:
        break
    frame = cv2.resize(frame0, None, fx=scaling_factor, fy=scaling_factor)
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    if first_frame:
        opt_flow = cv2.calcOpticalFlowFarneback(prev_frame, gray, None, 0.5, 5, 13, 10, 5, 1.1, cv2.OPTFLOW_FARNEBACK_GAUSSIAN)
        first_frame = False
    else:
        opt_flow= cv2.calcOpticalFlowFarneback(prev_frame, gray, opt_flow, 0.5, 5, 13, 10, 5, 1.1, cv2.OPTFLOW_USE_INITIAL_FLOW)
    prev_frame = np.copy(gray)

    if display_flow(frame, opt_flow):
        break;

    cv2.imshow("find_two_odds", frame)
    k = cv2.waitKey(1)
    if k == 27:
        break
        
cap.release()
cv2.destroyAllWindows()
'''Movement too small hard to detect'''
'''Mouse moving around, interfere flow result'''
'''Odd mii detection happens a while later, please wait for a moment'''
'''reminder: requires haarcascade_frontalface_default.xml'''

'Odd mii detection happens a while later, please wait for a moment'

###### %%writefile test
## Final Exam
<b>The general objective is to solve four different games to find mii.<br>
The code for each game should be written in an individual cell.<br>
Add comments in your code to explain your approach.<br>
You can design your own approach, and use any method you learned in this class.</b><br>

1A. Input images from video file WiiPlay.mp4 with level 15 (frame number between 4820 and 5000).<br> 
1B. (10pts) Acquire a <b>face template</b> from the first frame (frame number = 4820).<br>
1C. (10pts) Try to detect the face the same as the template on subsequent frames, draw a <b>red</b> rectangle around the detected face, and show the output images in the <b>"find_this_mii"</b> window.<br><br>

2A. Input images from video file WiiPlay.mp4 with level 8 (frame number between 2180 and 2380).<br>
2B. (10pts) Detect <b>pedestrians</b> on each frame and draw a <b>green</b> rectangle around your detection.<br>
2C. (10pts) Detect <b>faces</b> on each frame and draw a <b>blue</b> rectangle around your detection.<br>
2D. (10pts) Try to find two faces look alike each other, draw a <b>red</b> rectangle around each of the two faces, and show the output images in the <b>"find_two_look_alike"</b> window.<br><br>

3A. Input images from video file WiiPlay.mp4 with level 9 (frame number between 2480 and 2600).<br>
3B. (10pts) <b>Detect </b>faces(or pedestrians) on the first frame and draw a <b>blue</b> rectangle around your detection.<br>
3C. (10pts) <b>Track </b>faces(or pedestrians) on subsequent frames and draw a <b>green</b> rectangle around your tracking.<br>
3D. (<u>10pts bonus</u>) Try to find out the fastest character, draw a <b>red</b> rectangle around the fastest character, and show the output images in the <b>"find_the_fastest_character"</b> window.<br><br>

4A. Input images from video file WiiPlay.mp4 with level 6 (frame number between 1650 and 1800).<br>
4B. (10pts) Compute and show <b>optical flows</b> on each frame using <b>blue</b> arrows.<br>
4C. (<u>10pts bonus</u>) Try to detect two odd character who face the opposite direction from everyone else, draw a <b>red</b> rectangle around each of the two character, and show the output images in the <b>"find_two_odds"</b> window.<br><br>

5. (10pts) Any comments regarding the final exam? Which steps you believe you have completed? Which steps bother you?<br>
6. (5pts) Any suggestion to teaching assistants to improve this class?<br>
7. (5pts) Any suggestion to teacher to improve this class?<br>
8. Upload your Jupyter code file (*.ipynb). The submission deadline is <b>midnight on 1/8</b>.<br>

## Comments
5a. For <b>"find_this_mii"</b>, I completed all of it and I think that the template selection is critical to the detection, so please select the template again if the result is not satisfying.

5b. For <b>"find_two_look_alike"</b>, I detected the pedestrians and the faces, however it is difficult to accurately detect all faces as the training dataset only includes human faces not cartoonish faces. Therefore, I have to tweak the scaleFactor and minNeighbors of face_cascade.detectMultiScale to a very low threshold, which invited some unwanted results such as false positive of the road surface. This in turn affects my template matching of look alike faces as I use the detected faces as my face templates. Therefore, I implemented face detection with 2 different criteria. To detect faces for face templates, I added hsv skin colour threshold to compensate the low threshold of face_cascade.detectMultiScale and remove unwanted false positives. Then, I do face detection again but with weaker criteria and the detection results are in light blue boxes.

5c. For <b>"find_the_fastest_character"</b>, I completed all the steps, I chose pedestrians detection instead of faces and the reason is as stated above. Still, not all pedestrains are detected even with a low threshold. MedianFlow tracking is used to track them and I've successfully captured the fastest moving mii.

5d. For <b>"find_two_odds"</b>, I calculated the optical flows densely to improve my turning direction detection result but the side effect is that the program runs slowly. I try to detect the odd miis using optical flow vectors but the information on the target faces is only available when turning head, so the detection of odd miis only happens during that short time period. I thought of using the positions of eyes to determine the facing direction, however the eye detection algorithm doesn't work well on some cartoonish eyes. 

Additional: I've noticed different output results when the code is run on different computers, especially cell 2 and cell 3.

6. I hope that the practices and homework are marked and checked as soon as possible. Also, I think that it is better to write the deadline more accurately like before midnight instead of before 5pm to prevent confusion? I thought teacher said that the uploads are still counted as long as it is submitted during that day.

7. The practices and homework are great for understanding the usage of different techniques in computer vision, however some students who are weak in programming, especially those who only learn about Python in the class are struggling to come up with their own program code. My suggestion is that after the submission deadline of practices or homework, can there be a basic sample code about the practices or homework for reference? So that the students have an idea about the code implementation, this may better prepare them for the exams.